In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd
from optimize_hexs0 import evaluate, eval_q
import multiprocessing as mp
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
import json

In [ ]:
hs = 200
network = 'old'

path = f"data/metrostl/{network}"
with open(f"{path}/headways.json", 'r') as f:
    current_headways = json.load(f)

df_route_lens = pd.read_csv(f"{path}/processed/route_lens.csv", index_col="route_id")
base_drivers = {k: df_route_lens.loc[k].trip_len / v for k, v in current_headways.items()}
base_sum = sum(base_drivers.values())
drivers = {k:base_sum/len(base_drivers) for k in base_drivers} # All routes are the same to start
base_sum

In [ ]:
%%time
evaluate(base_drivers, hs=hs, max_dist=7200, orig='POP21', dest='C000', pct=True, scale=True, network=network)

In [ ]:
%%time
res_old = evaluate(base_drivers, hs=hs, max_dist=7200, orig='POP21', dest='C000', pct=True, scale=True, vis=True, network=network)

In [ ]:
res_old.plot('reached', legend=True, figsize=(20,15))
res_old.plot('reached_scaled', legend=True, figsize=(20,15), vmax=300)
# plt.savefig('test.png', bbox_inches='tight')
# plt.close()

In [ ]:
hs = 200
network = 'new'

path = f"data/metrostl/{network}"
with open(f"{path}/headways.json", 'r') as f:
    current_headways = json.load(f)

df_route_lens = pd.read_csv(f"{path}/processed/route_lens.csv", index_col="route_id")
base_drivers = {k: df_route_lens.loc[k].trip_len / v for k, v in current_headways.items()}
base_sum = sum(base_drivers.values())
drivers = {k:base_sum/len(base_drivers) for k in base_drivers} # All routes are the same to start
base_sum

In [ ]:
%%time
res_new = evaluate(base_drivers, hs=hs, max_dist=7200, orig='POP21', dest='C000', pct=True, scale=True, vis=True, network=network)

In [ ]:
res_new.plot('reached', legend=True, figsize=(20,15))
res_new.plot('reached_scaled', legend=True, figsize=(20,15), vmax=300)
# plt.savefig('test.png', bbox_inches='tight')
# plt.close()

In [ ]:
%%time
res_new['reached_diff'] = res_new.reached - res_old.reached
res_new['reached_scaled_diff'] = res_new.reached_scaled - res_old.reached_scaled

In [ ]:
res_new.plot('reached_scaled', legend=True, figsize=(20,15), vmax=300)
res_new.plot('reached_scaled_diff', legend=True, figsize=(20,15), vmax=50, vmin=-50, cmap='PiYG')
# plt.savefig('test.png', bbox_inches='tight')
# plt.close()